In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature = 0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

llm = OpenAI(model_name="gpt-3.5-turbo-1106")

a = llm.predict("how many planets are in the solar system?")
b = chat.predict("how many planets are in the solar system?")

promt = template.format(country_a="France", country_b="Germany")

chat.predict(promt)

#a,b


/Users/yoon/Documents/5.Ready/Fullstack-gpt/TheThird/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/yoon/Documents/5.Ready/Fullstack-gpt/TheThird/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'The distance between France and Germany varies depending on the specific locations being compared. The straight-line distance between Paris, France and Berlin, Germany is approximately 876 kilometers (544 miles). However, the driving distance between the two countries can be longer due to the road network and border crossings.'

In [2]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human","What is the distance between {country_a} and {country_b}?. Also, what is your name?")
])

prompt = template.format_messages(
    language = "Greek",
    name =  "Athena",
    country_a = "France",
    country_b = "Germany"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ Γαλλίας και Γερμανίας είναι περίπου 500 χιλιόμετρα. Το όνομά μου είναι Αθηνά. Πώς μπορώ να βοηθήσω;')

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage


messages = [
    SystemMessage(content= "You are a geography expert. And you only reply in {language}."),
    AIMessage(content= "Ciao, mi chiamo {name}!"),
    HumanMessage(content= "What is the distance between {country_a} and {country_b}?. Also, What is your name?")
]
chat.predict_messages(messages)

AIMessage(content='Mi dispiace, non posso fornire informazioni sulla distanza tra due paesi specifici. Il mio nome è {name}. Come posso aiutarti oggi?')

In [4]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        item = text.strip().split(",")
        return list(map(str.strip, item))

p = CommaOutputParser()
p.parse("Hello,how,are,you")



['Hello', 'how', 'are', 'you']

In [5]:
from langchain.schema import BaseOutputParser

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['mercury', 'venus', 'earth', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune']

In [6]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']